<a href="https://colab.research.google.com/github/crazair/Interesting-java-examples/blob/master/Lecture4_Clustering%26MLSD/PT_Practice4_WhoIsTalking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Занятие 4. Who’s Talking? – Classify Encrypted TCP Traffic

Сегодня вам предстоит работать с реальным зашифрованным корпоративным сетевым трафиком.
Ваша задача — классифицировать, какое приложение или сервис сгенерировал данный TCP-поток, **несмотря на то, что полезная нагрузка зашифрована**, и доступны только размеры пакетов и их направления.

Датасет был собран в контролируемой корпоративной среде, где каждой сетевой сессии было присвоено соответствующее приложение или сервис.
Каждая запись представляет собой усечённый TCP-поток длиной до 30 пакетов и содержит только информацию о длине пакетов и направлении передачи.
Такой формат позволяет исследовать, как методы машинного обучения могут выявлять поведенческие паттерны в зашифрованном трафике без анализа его содержимого.

### Описание данных

Данные содержат размеченные примеры TCP-потоков с соответствующим приложением или сервисом (`app_service`).

**Колонки:**

* **app_service** — целевая переменная: название или числовой идентификатор приложения или сервиса, сгенерировавшего TCP-поток (например, Telegram, YouTube, Zoom)

* **tcp_len_1 – tcp_len_30** — последовательность длин до 30 TCP-пакетов внутри потока.
- Положительные значения соответствуют пакетам, отправленным клиентом.
- Отрицательные значения соответствуют пакетам, полученным от сервера.
-  Если сессия содержит меньше 30 пакетов, оставшиеся позиции заполняются нулями.

# <font color="green">Осознание целей и ограничений бизнеса - вопросы для обсуждения с заказчиком</font>

### Бизнес и цель

- Для чего нужна классификация трафика?

Примеры: мониторинг использования приложений, защита корпоративной сети, аудит использования SaaS, оптимизация пропускной способности.

- Какие приложения/сервисы нас интересуют? Только конкретные ключевые (Zoom, YouTube, Telegram), или все возможные?

- Какое требование к точности классификации? Например, достаточно 80% или нужно >95%?


### Данные и сбор

- Насколько репрезентативен датасет по сравнению с реальностью?

Примеры: различное время суток, разные сети (Wi-Fi, проводная), разные пользователи.

- Как часто появляются новые приложения или обновления существующих? Как быть с классами «новых» приложений?

- Есть ли ограничения на доступ к метаданным трафика, кроме длины пакетов и направления?


### Ограничения на модель

- Время отклика: нужен ли реальный онлайн детектор для каждого потока или можно пакетно/батчево?

- Ресурсы: допустимая нагрузка на CPU/GPU, память, скорость обработки.

- Частота обновления модели: сколько раз в неделю/месяц будем переобучать?


### Конфиденциальность и безопасность

- Данные трафика зашифрованы. Нужно ли дополнительно анонимизировать метаданные или учитывать регуляторные ограничения?

# <font color="green">Особенности данных и подготовка </font>

### Формат данных

- Последовательности TCP-пакетов длиной до 30, с положительными/отрицательными значениями.

- Пустые позиции заполнены нулями.

- Целевая переменная — app_service.


### Возможные pre-processing шаги

- Нормализация длин пакетов (например, деление на 1500 для приведения к [−1,1]).


### Дополнительные признаки

- Статистика по пакету: средняя длина, медиана, стандартное отклонение.

- Количество пакетов в каждом направлении (up/down).

- Интервалы между пакетами (если доступны временные метки).

- Можно использовать sequence models (RNN, LSTM, Transformer) или feature-based models (Random Forest, XGBoost).


### Особенности

- Потоки разной длины → padding до 30.

- Потоки сильно разрежены → много нулей.

- Много приложений → возможно сильный class imbalance.

# <font color="green">Бизнес и офлайн-метрики </font>

### Бизнес-метрики

- Доля правильно классифицированных приложений в онлайн-сценарии.

- Доля ошибочной классификации для критичных приложений (Zoom, корпоративные SaaS).

- Влияние на решения по безопасности и мониторингу (например, блокировка нежелательных сервисов).


### Офлайн ML-метрики

- Accuracy / Macro F1 / Weighted F1 (особенно при несбалансированных классах)

- Confusion matrix → какие приложения путаются чаще всего

- Top-K accuracy (например, Top-3, если нужно знать несколько вероятных кандидатов)

- ROC-AUC для каждого класса (multi-class AUC)

- Precision/Recall для ключевых приложений (например, высокочастотных или критичных)


### Особые моменты для офлайн-метрик

- Обратить внимание на имитацию реальной среды: train/test split должен быть по сессиям и пользователям, чтобы модель не запоминала конкретные шаблоны отдельных пользователей.

Сначала скачаем данные

In [49]:
# Изменение 0. Делаем удобную загрузку данных с проверками в едином блоке

import json
import os

!wget -O response.json "https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/7LCvNsGL1mv90Q"

with open("response.json") as f:
    data = json.load(f)

href = data["href"]
out = "archive.zip"
print(href)

if os.path.exists(out):
    print(f"{out} уже существует — пропускаю скачивание.")
else:
    !wget -O "{out}" "$href"
    !unzip archive.zip -d data

--2026-02-27 16:30:34--  https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://disk.yandex.ru/d/7LCvNsGL1mv90Q
Resolving cloud-api.yandex.net (cloud-api.yandex.net)... 213.180.204.127, 2a02:6b8::1:127
Connecting to cloud-api.yandex.net (cloud-api.yandex.net)|213.180.204.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 617 [application/json]
Saving to: ‘response.json’

response.json       100%[===================>]     617  --.-KB/s    in 0s      

2026-02-27 16:30:35 (39.5 MB/s) - ‘response.json’ saved [617/617]

https://downloader.disk.yandex.ru/disk/40cab9cd9c250e64b525caa5329173d473d3aa678fa60542c5f20f5571567f9e/69a1feeb/fKqInKw3d7bLFOeFnMGnhOEt74e_AcHrfoeOiWQFqnxSx2KlJFe3moqi-rTFCO6djYu3q6EHAUP4X1BQu7yk7eyLDbl9lafVd4ruaDAPlxGr8npumZHI4midPdWhecNq?uid=0&filename=whos-talking-classify-the-app-by-its-packets.zip&disposition=attachment&hash=m8VLCd1u1YCGUik9dZVkJjbxXRNbrE0kS1p%2BEIBguw1klnm56L9GDaExZ6Yk0MeDq/J6bpmRyOJonT3VoXnDag%3

Установим необходимые библиотеки

In [50]:
!pip install catboost -q

import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool

Загрузим данные и посмотрим на них

In [51]:
df = pd.read_csv('/content/data/train.csv')

df.info()

/tmp/ipython-input-20418/525312853.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/data/train.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248546 entries, 0 to 8248545
Data columns (total 31 columns):
 #   Column       Dtype  
---  ------       -----  
 0   app_service  object 
 1   tcp_len_1    int64  
 2   tcp_len_2    float64
 3   tcp_len_3    float64
 4   tcp_len_4    float64
 5   tcp_len_5    float64
 6   tcp_len_6    float64
 7   tcp_len_7    float64
 8   tcp_len_8    float64
 9   tcp_len_9    float64
 10  tcp_len_10   float64
 11  tcp_len_11   float64
 12  tcp_len_12   float64
 13  tcp_len_13   float64
 14  tcp_len_14   float64
 15  tcp_len_15   float64
 16  tcp_len_16   float64
 17  tcp_len_17   float64
 18  tcp_len_18   float64
 19  tcp_len_19   float64
 20  tcp_len_20   float64
 21  tcp_len_21   float64
 22  tcp_len_22   float64
 23  tcp_len_23   float64
 24  tcp_len_24   float64
 25  tcp_len_25   float64
 26  tcp_len_26   float64
 27  tcp_len_27   float64
 28  tcp_len_28   float64
 29  tcp_len_29   float64
 30  tcp_len_30   float64
dtypes: float64(29), int64(1)

In [52]:
df.sample(5)

,app_service,tcp_len_1,tcp_len_2,tcp_len_3,tcp_len_4,tcp_len_5,tcp_len_6,tcp_len_7,tcp_len_8,tcp_len_9,...,tcp_len_21,tcp_len_22,tcp_len_23,tcp_len_24,tcp_len_25,tcp_len_26,tcp_len_27,tcp_len_28,tcp_len_29,tcp_len_30
2282477,Skype,1440,774.0,-99.0,1060.0,-1460.0,-1460.0,-1460.0,-1460.0,-228.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3620765,azure,1410,398.0,-99.0,622.0,-1410.0,-1410.0,-1276.0,-1410.0,-1410.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5953077,mts,1380,377.0,-1380.0,-1380.0,-1380.0,-1324.0,-1380.0,-1380.0,-1380.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
754924,4,1448,336.0,-1448.0,-1116.0,64.0,1208.0,673.0,-1208.0,-1448.0,...,-689.0,-1413.0,-1448.0,-1448.0,-1448.0,-1242.0,-1448.0,-1448.0,-1448.0,-1448.0
5358866,lichess,499,-1308.0,-1308.0,-1238.0,64.0,1037.0,-1208.0,-1308.0,-1086.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
df.memory_usage(deep=True).sum() / 1024**2

np.float64(2317.443407058716)

Казалось бы по определению длина - целая виличина, а тут используеться float64, давайте экономить память. Проверим диапозон значений, найдем есть ли наны или инфы, проверим количество не целых чисел

In [54]:
target_col = "app_service"
feat_cols = [c for c in df.columns if c.startswith("tcp_len_")]

In [55]:
stats = {
    "min_value": df[feat_cols].min().min(),
    "max_value": df[feat_cols].max().max(),
    "nan_count": np.isnan(df[feat_cols].values).sum(),
    "inf_count": np.isinf(df[feat_cols].values).sum(),
    "non_integer_count": np.sum((df[feat_cols].values % 1) != 0),
}

stats

{'min_value': -1464.0,
 'max_value': 1460.0,
 'nan_count': np.int64(0),
 'inf_count': np.int64(0),
 'non_integer_count': np.int64(0)}

Как видно значения по в диапозоне от -1464 до 1460 нет ни нанов ни инфов ни дробных чисел - изменим тип на int16

In [56]:
feat_cols = [c for c in df.columns if c.startswith("tcp_len_")]
df[feat_cols] = df[feat_cols].astype(np.int16)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8248546 entries, 0 to 8248545
Data columns (total 31 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   app_service  object
 1   tcp_len_1    int16 
 2   tcp_len_2    int16 
 3   tcp_len_3    int16 
 4   tcp_len_4    int16 
 5   tcp_len_5    int16 
 6   tcp_len_6    int16 
 7   tcp_len_7    int16 
 8   tcp_len_8    int16 
 9   tcp_len_9    int16 
 10  tcp_len_10   int16 
 11  tcp_len_11   int16 
 12  tcp_len_12   int16 
 13  tcp_len_13   int16 
 14  tcp_len_14   int16 
 15  tcp_len_15   int16 
 16  tcp_len_16   int16 
 17  tcp_len_17   int16 
 18  tcp_len_18   int16 
 19  tcp_len_19   int16 
 20  tcp_len_20   int16 
 21  tcp_len_21   int16 
 22  tcp_len_22   int16 
 23  tcp_len_23   int16 
 24  tcp_len_24   int16 
 25  tcp_len_25   int16 
 26  tcp_len_26   int16 
 27  tcp_len_27   int16 
 28  tcp_len_28   int16 
 29  tcp_len_29   int16 
 30  tcp_len_30   int16 
dtypes: int16(30), object(1)
memory usage: 534.9+ MB


In [57]:
df.memory_usage(deep=True).sum() / 1024**2

np.float64(901.4866428375244)

Почти в 4 раза экономнее, RAM в колабе скажет спасибо

In [58]:
N = len(df)
N

8248546

In [59]:
X0 = df[feat_cols]
y = df[target_col].astype(str)

sample_n = 1_000_000 # всё не влезет
idx = np.random.RandomState(42).choice(N, size=sample_n, replace=False)

X0 = X0.iloc[idx]
y  = y.iloc[idx]

Посмотрим на целевую переменную

In [60]:
y.value_counts()

,count
app_service,
Telegram,12315
287,12277
GMail,12277
Wildberries,12268
Teams,12266
...,...
apteka-ru,135
appdynamics,133
MEGA,131


In [61]:
set(y)

{'1',
 '1-password',
 '101-radio',
 '113',
 '120',
 '15',
 '218',
 '287',
 '299',
 '2gis',
 '343',
 '4',
 '93',
 'Amazon',
 'Anydesk',
 'Apple',
 'Baidu',
 'DataSaver',
 'DoH_DoT',
 'DropBox',
 'Facebook',
 'GMail',
 'Github',
 'Gitlab',
 'Instagram',
 'Ktalk',
 'LinkedIn',
 'Livejournal',
 'MEGA',
 'MSN',
 'Mattermost',
 'Odnoklassniki',
 'Ozon',
 'PlayStore',
 'Playstation',
 'QQ',
 'Quora',
 'Reddit',
 'Rutube',
 'Skype',
 'Snapchat',
 'SoundCloud',
 'Spotify',
 'Steam',
 'Teams',
 'Telegram',
 'TikTok',
 'Twitch',
 'Twitter',
 'WhatsApp',
 'Wikipedia',
 'Wildberries',
 'Yandex',
 'YouTube',
 'adblock-plus',
 'adobe',
 'aeroflot-online',
 'afisha-ru',
 'alfabank',
 'ali-wangwang-file-transfer',
 'alipay',
 'amd-online',
 'andata',
 'any-run',
 'anygo',
 'appdynamics',
 'apple-community',
 'apple-siri',
 'apple_icloud',
 'apple_push',
 'apteka-ru',
 'avito',
 'azure',
 'banki-ru',
 'beeline-online',
 'bing',
 'bitly',
 'blog-posting',
 'braintree',
 'bugsnag',
 'bybit',
 'ca-technolo

### Feature engineering

In [62]:
# Изменение 2. Фичинжиниринг. Добавялем новые признаки. Описания в них.
# val accuracy: 0.814175 ->

X = X0.copy()

pos    = (X0 > 0)
neg    = (X0 < 0)
nonzero = (X0 != 0)

# Базовые признаки
X["pkt_cnt"]    = nonzero.sum(axis=1).astype(np.int16)
X["n_up"]       = pos.sum(axis=1).astype(np.int16)
X["n_down"]     = neg.sum(axis=1).astype(np.int16)
X["bytes_up"]   = X0.where(pos, 0).sum(axis=1).astype(np.int32)
X["bytes_down"] = (-X0.where(neg, 0)).sum(axis=1).astype(np.int32)

# Вспомогательные матрицы
abs_nonzero = X0.abs().where(nonzero, np.nan)   # abs значения, нули → NaN
up_vals     = X0.where(pos, np.nan)             # только клиентские пакеты
down_vals   = (-X0).where(neg, np.nan)          # только серверные пакеты (положит.)

# Соотношения up/down
X["ratio_up_down"] = (X["n_up"] / (X["n_down"] + 1)).clip(0, 30)
X["ratio_bytes"]   = (X["bytes_up"] / (X["bytes_down"] + 1)).clip(0, 100)

# Статистики по размерам пакетов
X["mean_pkt_size"] = abs_nonzero.mean(axis=1).fillna(0)
X["std_pkt_size"]  = abs_nonzero.std(axis=1).fillna(0)
X["max_pkt_size"]  = abs_nonzero.max(axis=1).fillna(0)
X["min_pkt_nz"]    = abs_nonzero.min(axis=1).fillna(0)

# Средний размер пакета отдельно вверх / вниз
X["mean_up"]   = up_vals.mean(axis=1).fillna(0)
X["mean_down"] = down_vals.mean(axis=1).fillna(0)
X["std_up"]    = up_vals.std(axis=1).fillna(0)    # новое: разброс клиентских
X["std_down"]  = down_vals.std(axis=1).fillna(0)  # новое: разброс серверных

# Первые 5 пакетов — handshake паттерн
first_cols = feat_cols[:5]
X["first5_sum"]    = X0[first_cols].sum(axis=1)
X["first5_sign"]   = pos[first_cols].sum(axis=1)   # сколько из первых 5 — клиентские
X["first_pkt_dir"] = np.sign(X0[feat_cols[0]])     # направление самого первого пакета

# Последние 5 пакетов — финальный паттерн потока
last_cols = feat_cols[-5:]
X["last5_sum"]  = X0[last_cols].sum(axis=1)
X["last5_sign"] = pos[last_cols].sum(axis=1)

# Смены направления
vals = X0.values
dir_sw = np.zeros(len(X0), dtype=np.int16)
for i in range(len(vals)):
    seq = vals[i][vals[i] != 0]
    if len(seq) > 1:
        dir_sw[i] = (np.diff(np.sign(seq)) != 0).sum()
X["direction_switches"] = dir_sw

# Удаляем бесполезные признаки
X = X.drop(columns=['tcp_len_29', 'tcp_len_25', 'tcp_len_26', 'tcp_len_22', 'tcp_len_28', 'tcp_len_30', 'tcp_len_21', 'tcp_len_23', 'tcp_len_24', 'first_pkt_dir', 'last5_sign'])


print(f"Итого признаков: {X.shape[1]}")
X.head()

Итого признаков: 40


,tcp_len_1,tcp_len_2,tcp_len_3,tcp_len_4,tcp_len_5,tcp_len_6,tcp_len_7,tcp_len_8,tcp_len_9,tcp_len_10,...,max_pkt_size,min_pkt_nz,mean_up,mean_down,std_up,std_down,first5_sum,first5_sign,last5_sum,direction_switches
3899906,517,-1448,-1448,-1448,-23,64,992,-179,-62,-31,...,1448.0,23.0,402.866667,392.200000,516.585751,556.558199,-3850,1,312,14
5090771,543,-399,51,234,-238,-1460,-1460,-1209,251,-1405,...,1460.0,7.0,258.375000,1094.227273,133.749166,562.322813,191,3,-3982,13
6160203,517,-203,314,-1460,-349,-1136,250,-1391,-1137,250,...,1460.0,24.0,332.750000,914.000000,126.484189,561.011809,-1181,2,0,7
3764889,517,-1448,-1448,-579,64,1208,56,-1208,-1448,-946,...,1448.0,56.0,735.333333,1119.500000,601.589783,402.416273,-2894,2,0,6
7759903,1460,647,-1460,-1460,-1176,-1287,64,92,548,-287,...,1460.0,24.0,413.428571,834.363636,527.616609,595.353218,-1989,2,0,5


### Обучение модели и валидация

In [63]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [64]:
# Изменение 1. Увеличить количество итераций и глубину дерева
# Самое очевидное — модель просто не успела обучиться. На 9-й итерации accuracy всё ещё росла.
# val accuracy: 0.447595 -> 0.814175

model = CatBoostClassifier(
    loss_function="MultiClass",  # MultiClass используется для задач классификации с 3+ классами (Softmax под капотом).
    eval_metric="Accuracy",      # Метрика, по которой оценивается качество модели на eval_set (валидационной выборке). "Accuracy" — доля правильно классифицированных примеров.
    iterations=300,              # Количество деревьев (шагов бустинга), которые модель построит. Было 10 — модель явно не сошлась
    depth=8,                     # Максимальная глубина каждого дерева решений. Было 5 — деревья слишком мелкие для многоклассовой задачи. Допустимые значения для CatBoost: 1–16.
    learning_rate=0.1,           # Шаг обучения (скорость градиентного спуска). Обычный диапазон: 0.01 – 0.3. Меньше → обучение медленнее, но точнее; больше → быстрее, но риск "перепрыгнуть" минимум.
    random_seed=42,              # Фиксирует генератор случайных чисел для воспроизводимости результатов.
    verbose=10,                  # Управляет выводом лога обучения в консоль. True — выводить каждую итерацию. Можно передать целое число, например verbose=50 — выводить каждые 50 итераций.
    task_type="GPU",             # Устройство, на котором будет происходить обучение.
    devices="0",                 # Какую именно видеокарту использовать (по индексу).
    early_stopping_rounds=30,    # остановка, если нет улучшений 30 итераций подряд
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True
)

pred = model.predict(X_val).reshape(-1)
print("val accuracy:", accuracy_score(y_val, pred))

"""
# Смотрим — какие фичи реально помогают, а какие мусорят
feat_imp = pd.DataFrame({
    "feature":    X_train.columns,
    "importance": model.get_feature_importance()
}).sort_values("importance", ascending=False)

print(feat_imp.to_string(index=False))

# Признаки с нулевой важностью — кандидаты на удаление
# Если какая-то фича окажется в useless — убираем, это уменьшит шум и улучшит accuracy
useless = feat_imp[feat_imp["importance"] < 0.01]["feature"].tolist()
print(f"\nБесполезные признаки: {useless}")
"""

0:	learn: 0.3027063	test: 0.3022700	best: 0.3022700 (0)	total: 5.33s	remaining: 26m 32s
10:	learn: 0.5621150	test: 0.5606600	best: 0.5606600 (10)	total: 58.8s	remaining: 25m 43s
20:	learn: 0.6629763	test: 0.6609850	best: 0.6609850 (20)	total: 1m 52s	remaining: 24m 53s
30:	learn: 0.7164038	test: 0.7138400	best: 0.7138400 (30)	total: 2m 46s	remaining: 24m
40:	learn: 0.7526312	test: 0.7488400	best: 0.7488400 (40)	total: 3m 38s	remaining: 22m 59s
50:	learn: 0.7777550	test: 0.7740450	best: 0.7740450 (50)	total: 4m 30s	remaining: 22m 1s
60:	learn: 0.7964500	test: 0.7921200	best: 0.7921200 (60)	total: 5m 24s	remaining: 21m 10s
70:	learn: 0.8102312	test: 0.8056750	best: 0.8056750 (70)	total: 6m 16s	remaining: 20m 14s
80:	learn: 0.8215050	test: 0.8163600	best: 0.8163600 (80)	total: 7m 8s	remaining: 19m 17s
90:	learn: 0.8339700	test: 0.8286550	best: 0.8286550 (90)	total: 8m 1s	remaining: 18m 25s
100:	learn: 0.8429562	test: 0.8377600	best: 0.8377600 (100)	total: 8m 52s	remaining: 17m 29s
110:	lea

'\n# Смотрим — какие фичи реально помогают, а какие мусорят\nfeat_imp = pd.DataFrame({\n    "feature":    X_train.columns,\n    "importance": model.get_feature_importance()\n}).sort_values("importance", ascending=False)\n\nprint(feat_imp.to_string(index=False))\n\n# Признаки с нулевой важностью — кандидаты на удаление\n# Если какая-то фича окажется в useless — убираем, это уменьшит шум и улучшит accuracy\nuseless = feat_imp[feat_imp["importance"] < 0.01]["feature"].tolist()\nprint(f"\nБесполезные признаки: {useless}")\n'

# <font color="green">Внедрение и deployment </font>


### Онлайн vs оффлайн

- Онлайн: классификация каждого TCP-потока на лету → низкая задержка, нужна оптимизация модели.

- Оффлайн: анализ собранных логов → более тяжёлые модели, batch processing.


### Архитектура

- Data collector → потоковые данные о TCP → Preprocessing → Model → Results

- Возможна интеграция с корпоративным мониторингом.


### Выход модели

- Прямое приложение/сервис

- Top-K вероятности для учета неопределённости

- Возможность отбрасывать низкоуверенные предсказания.

# <font color="green">Логгирование и мониторинг </font>


### Что логировать

- Входные потоки (только метаданные, без payload)

- Предсказанные классы и вероятности

- Ошибки классификации (если есть метки)

- Время обработки


### Мониторинг

- Performance drift: точность со временем, особенно при обновлениях приложений

- Data drift: изменение распределения длины пакетов и направлений

- Модельный лог: уверенность предсказания, частота каждого класса

- Alerts: на резкие изменения трафика или появление неизвестных приложений


### Метрики производительности

- Latency per flow

- Throughput (flows/sec)

- Resource usage (CPU, RAM)